[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [1]:
import requests
import base64
from copy import deepcopy
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
from bs4 import BeautifulSoup as bsoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import ast
import emoji

client_key = "uDxiPfIPIcFjGD0fmxj7c2XAk"
client_secret = "Bcr55JD2dNhGFftvtWwn19x78pRPLNP5rCBmGw0yVeW5Uy3hpc" 

In [2]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAEskIQEAAAAAqkugIbkkPuB3XUUBhHAYkbYX4ik%3DjxVjLVRb5O9OUZ95hy8Z3cDHaTkA4j7VJM0Shx3GNLq2W8SdOA'}

In [6]:
access_token = json_data['access_token']

# Get Tweets
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=1000
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [8]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

keywords = ["Packers","Bears","Eagles", "Cowboys","Raiders", "Chiefs"]

for keyword in keywords:

    parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit, 
                     'tweet_mode': 'extended'}

    search_url = base_url+'1.1/search/tweets.json'

    response = requests.get(search_url, headers=search_headers, params=parameters)
    football_tweets = response.json()

    for status in football_tweets['statuses']:
        status["team"] = keyword
    tweets += football_tweets['statuses']
    ids = [tw['id'] for tw in football_tweets['statuses']]
    min_id = min(ids)
    tw_ids = []
    tw_ids+=ids

    for  i in range(max_tweets//tw_limit -1):
        parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit,
                    'max_id': min(tw_ids)-1
                     }
        print("searching tweets with id  < {}".format(min_id))
        search_url = base_url+'1.1/search/tweets.json'
        response = requests.get(search_url, headers=search_headers, params=parameters)
        football_tweets = response.json()
        ids = [tw['id'] for tw in football_tweets['statuses']]
        tw_ids+=ids
        print(i, len(football_tweets['statuses']))
        min_id = min(ids)
        for status in football_tweets['statuses']:
            status["team"] = keyword
        tweets += football_tweets['statuses']    

searching tweets with id  < 1326526104753102852
0 100
searching tweets with id  < 1326519389768871936
1 100
searching tweets with id  < 1326509499683909633
2 100
searching tweets with id  < 1326497415260741633
3 100
searching tweets with id  < 1326478927808884736
4 100
searching tweets with id  < 1326453111033106433
5 100
searching tweets with id  < 1326429232948531200
6 100
searching tweets with id  < 1326411118420627457
7 100
searching tweets with id  < 1326399435371393026
8 100
searching tweets with id  < 1326539206856826883
0 100
searching tweets with id  < 1326538211045208071
1 100
searching tweets with id  < 1326537241489932290
2 100
searching tweets with id  < 1326536351018217472
3 100
searching tweets with id  < 1326535411628466176
4 100
searching tweets with id  < 1326534235331686402
5 100
searching tweets with id  < 1326533146825936897
6 100
searching tweets with id  < 1326531850060267521
7 100
searching tweets with id  < 1326530694139400193
8 100
searching tweets with id  < 

As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [9]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 6000


# Let's print some tweets

In [10]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [11]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Nov 12 22:09:12 +0000 2020',
  'display_text_range': [0, 170],
  'entities': {'hashtags': [],
               'symbols': [],
               'urls': [],
               'user_mentions': [{'id': 23642374,
                                  'id_str': '23642374',
                                  'indices': [56, 65],
                                  'name': 'Seattle Seahawks',
                                  'screen_name': 'Seahawks'},
                                 {'id': 35865630,
                                  'id_str': '35865630',
                                  'indices': [86, 94],
                                  'name': 'Green Bay Packers',
                                  'screen_name': 'packers'},
                                 {'id': 33583496,
                                  'id_str': '33583496',
                                  'indices': [114, 121],
                                  'name': 'Kansa

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [12]:
for status in tweets[0:max_tweet_to_print]:
    print(status['full_text'] + '\n')

Who was the MVP of the first half of the 2020 season?
- @Seahawks QB Russell Wilson
- @Packers QB Aaron Rodgers
- @Chiefs QB Patrick Mahomes
- @BuffaloBills QB Josh Allen

Honoring all those who have served &amp; continue to serve.

Thank you. 🇺🇸

#VeteransDay | #SaluteToService https://t.co/IHj4emCuEw

Davante Adams is closing in on that 99 overall 👀 @tae15adams @packers @ochocinco @EAMaddenNFL https://t.co/QiBXITHgk4

Any Colts or Packers fan that want to go to the Packers @ Colts next weekend? We got one ticket available
$450

Flights+room coming out to roughly $1k



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [13]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'team'])
28


In [14]:
import pandas as pd

# Generators for building DataFrames

In [15]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here

        modified_tweet = deepcopy(tweet)
        try:
            modified_tweet["user_id"] = modified_tweet["user"]["id"]
            modified_tweet.pop("user", None)
        except KeyError:
            pass
        try:
            modified_tweet["retweeted_status_id"] = modified_tweet["retweeted_status"]["id"] 
            modified_tweet.pop("retweeted_status", None)
        except KeyError:
            pass
        yield modified_tweet       
            
            

In [16]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'created_at': 'Thu Nov 12 22:09:12 +0000 2020',
  'id': 1327010583200337926,
  'id_str': '1327010583200337926',
  'full_text': 'Who was the MVP of the first half of the 2020 season?\n- @Seahawks QB Russell Wilson\n- @Packers QB Aaron Rodgers\n- @Chiefs QB Patrick Mahomes\n- @BuffaloBills QB Josh Allen',
  'truncated': False,
  'display_text_range': [0, 170],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'Seahawks',
     'name': 'Seattle Seahawks',
     'id': 23642374,
     'id_str': '23642374',
     'indices': [56, 65]},
    {'screen_name': 'packers',
     'name': 'Green Bay Packers',
     'id': 35865630,
     'id_str': '35865630',
     'indices': [86, 94]},
    {'screen_name': 'Chiefs',
     'name': 'Kansas City Chiefs',
     'id': 33583496,
     'id_str': '33583496',
     'indices': [114, 121]},
    {'screen_name': 'BuffaloBills',
     'name': 'Buffalo Bills',
     'id': 25084916,
     'id_str': '25084916',
     'indices': [143, 156]}],
   'u

In [17]:
football_feed_df = pd.DataFrame(tweet_generator(tweets))

# Text Cleanup

Here, we're going to take to start cleaning the text in the tweets.

In [18]:
print("Rows with either null text of teams: {}".format(football_feed_df[["full_text","team"]].isnull().any(axis = 1).sum())) 

Rows with either null text of teams: 5400


In [19]:
def NoneToNan(elem):
    if elem == "None" or elem == None:
        return np.nan
    else:
        return elem

football_feed_df = football_feed_df.applymap(NoneToNan)

In [20]:
football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,Who was the MVP of the first half of the 2020 ...,Packers,NaN
1,Honoring all those who have served &amp; conti...,Packers,NaN
2,Davante Adams is closing in on that 99 overall...,Packers,NaN
3,Any Colts or Packers fan that want to go to th...,Packers,NaN
4,RT @hernameisitzel: hahaha,Packers,NaN
...,...,...,...
5995,NaN,Chiefs,"Then after these (Messengers), We sent Moses w..."
5996,NaN,Chiefs,That Chiefs team fired its entire defensive co...
5997,NaN,Chiefs,"RT @EliteJuJu: So let me get this right, the S..."
5998,NaN,Chiefs,@RPatrickAllen @nj_chiefs @D_Harms19 I saw a n...


In [21]:
football_feed_df.full_text.fillna(football_feed_df.text, inplace=True)

football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,Who was the MVP of the first half of the 2020 ...,Packers,NaN
1,Honoring all those who have served &amp; conti...,Packers,NaN
2,Davante Adams is closing in on that 99 overall...,Packers,NaN
3,Any Colts or Packers fan that want to go to th...,Packers,NaN
4,RT @hernameisitzel: hahaha,Packers,NaN
...,...,...,...
5995,"Then after these (Messengers), We sent Moses w...",Chiefs,"Then after these (Messengers), We sent Moses w..."
5996,That Chiefs team fired its entire defensive co...,Chiefs,That Chiefs team fired its entire defensive co...
5997,"RT @EliteJuJu: So let me get this right, the S...",Chiefs,"RT @EliteJuJu: So let me get this right, the S..."
5998,@RPatrickAllen @nj_chiefs @D_Harms19 I saw a n...,Chiefs,@RPatrickAllen @nj_chiefs @D_Harms19 I saw a n...


In [22]:
hashtags = []
user_mentions = []

for row in football_feed_df.itertuples():
    row = row.entities
    try:
        hashtags_in_row = row["hashtags"]
        hashtags.append(hashtags_in_row)
    except:
        hashtags.append('')
    try:
        user_mentions_in_row = row["user_mentions"][0]["name"]
        user_mentions.append(user_mentions_in_row)
    except:
        user_mentions.append('')



In [23]:
football_feed_df["hashtags"] = hashtags
football_feed_df["user_mentions"] = user_mentions
football_feed_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,user_id,extended_entities,possibly_sensitive,retweeted_status_id,quoted_status_id,quoted_status_id_str,quoted_status,text,hashtags,user_mentions
0,Thu Nov 12 22:09:12 +0000 2020,1327010583200337926,1327010583200337926,Who was the MVP of the first half of the 2020 ...,False,"[0, 170]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://ads-api.twitter.com"" rel=""nof...",NaN,...,23508439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Seattle Seahawks
1,Wed Nov 11 14:04:03 +0000 2020,1326526104753102852,1326526104753102852,Honoring all those who have served &amp; conti...,False,"[0, 107]","{'hashtags': [{'text': 'VeteransDay', 'indices...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,35865630,"{'media': [{'id': 1326525134014996480, 'id_str...",False,NaN,NaN,NaN,NaN,NaN,"[{'text': 'VeteransDay', 'indices': [76, 88]},...",
2,Thu Nov 12 20:07:09 +0000 2020,1326979869754843136,1326979869754843136,Davante Adams is closing in on that 99 overall...,False,"[0, 93]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,67133275,"{'media': [{'id': 1326979396322775040, 'id_str...",False,NaN,NaN,NaN,NaN,NaN,[],Davante Adams
3,Fri Nov 13 03:04:21 +0000 2020,1327084862415638536,1327084862415638536,Any Colts or Packers fan that want to go to th...,False,"[0, 150]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,255454351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],
4,Fri Nov 13 03:04:01 +0000 2020,1327084776822624256,1327084776822624256,RT @hernameisitzel: hahaha,False,"[0, 26]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'tl', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,1214530451613859840,NaN,NaN,1.327084e+18,NaN,NaN,NaN,NaN,[],itzel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Wed Nov 11 17:19:50 +0000 2020,1326575373220540422,1326575373220540422,"Then after these (Messengers), We sent Moses w...",True,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.example.com"" rel=""nofollow...",NaN,...,794002277165993985,NaN,NaN,NaN,NaN,NaN,NaN,"Then after these (Messengers), We sent Moses w...",[],
5996,Wed Nov 11 17:19:47 +0000 2020,1326575363317669889,1326575363317669889,That Chiefs team fired its entire defensive co...,True,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://about.twitter.com/products/tw...",NaN,...,176606900,NaN,False,NaN,1.326575e+18,1326574660331442178,{'created_at': 'Wed Nov 11 17:17:00 +0000 2020...,That Chiefs team fired its entire defensive co...,[],
5997,Wed Nov 11 17:19:39 +0000 2020,1326575327657816066,1326575327657816066,"RT @EliteJuJu: So let me get this right, the S...",False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,1011176887,NaN,NaN,1.326551e+18,NaN,NaN,NaN,"RT @EliteJuJu: So let me get this right, the S...",[],chris ⓳
5998,Wed Nov 11 17:19:34 +0000 2020,1326575309030744064,1326575309030744064,@RPatrickAllen @nj_chiefs @D_Harms19 I saw a n...,True,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.326571e+18,...,1103747253049614336,NaN,NaN,NaN,NaN,NaN,NaN,@RPatrickAllen @nj_chiefs @D_Harms19 I saw a n...,[],Patrick Allen


In [24]:
# write code here

def clean_tweet(text):  

    url = re.compile('https://.+')
    rt = re.compile("RT")
    user = re.compile('@\S+\s')
 
    #'(RT|@.*\s|https://.+)'

    text = url.sub("", text)
    text = rt.sub("", text)
    text = user.sub("", text)

    return text

clean_tweet("RT .@SteveKingIA says he's unsure about the science of climate change. But you don’t need to be a scientist to take ac… https://t.co/J1CcrQZkB2")

football_feed_df["full_text"] = football_feed_df["full_text"].apply(clean_tweet)

In [26]:

def emoji_retriever(text):
      return [(char, emoji.demojize(char)) for char in text if char in emoji.UNICODE_EMOJI]

football_feed_df["emoji"] = football_feed_df["full_text"].apply(emoji_retriever)

In [27]:
contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
(r'haven\'t', 'have not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

def preprocess_text(text):
    
    contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
    (r'haven\'t', 'have not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'re', '\g<1> are')]

    replacer = contraction_replacer(contraction_patterns)
    text = replacer.do_contraction_normalization(text)
    text = nltk.tokenize.word_tokenize(text)

    stop_words = set(stopwords.words('english')) 
    
    # lemmatization
    wnlemma = WordNetLemmatizer()
    lemma_text = []
    for word in text:
        if word in stop_words:
            continue
        lemma_text.append(wnlemma.lemmatize(word))
    lemma_text = " ".join(lemma_text)

    return lemma_text

football_feed_df["processed_text"] = football_feed_df["full_text"].apply(preprocess_text)

In [28]:
#Pickle dataframe
import pickle
football_feed_df.to_pickle('football_feed_df.pkl')

In [29]:
def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

# Sentiment Analysis

Below represents the relative polarity of each team from each team on Twitter.

In [30]:
sentiments

team
Bears      0.124317
Chiefs     0.139599
Cowboys    0.239837
Eagles     0.219107
Packers    0.148294
Raiders    0.071166
dtype: float64

In [31]:
def tweet_user_generator(tweets):
    for tweet in tweets:
        yield tweet["user"]
                 

In [32]:
user_df = pd.DataFrame(tweet_user_generator(tweets))

In [33]:
user_df.head()

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,23508439,23508439,FOX Sports: NFL,NFLonFOX,,Download the new FOX Sports App now:,https://t.co/2qP5yLFi3y,{'url': {'urls': [{'url': 'https://t.co/2qP5yL...,False,1952567,...,EFEFEF,333333,True,False,False,False,None,None,None,none
1,35865630,35865630,Green Bay Packers,packers,"Green Bay, WI",The official Twitter account of the 13-time Wo...,http://t.co/SOkNpLbeMb,{'url': {'urls': [{'url': 'http://t.co/SOkNpLb...,False,2308661,...,DDFFCC,333333,False,True,False,False,None,None,None,none
2,67133275,67133275,The Checkdown,thecheckdown,,The best in the game. 🏈,https://t.co/fiY5l74YJT,{'url': {'urls': [{'url': 'https://t.co/fiY5l7...,False,411058,...,DDFFCC,333333,False,False,False,False,None,None,None,none
3,255454351,255454351,Kevin S,Mainstreetkills,"Los Angeles, CA",,None,{'description': {'urls': []}},False,302,...,DDEEF6,333333,True,True,False,False,None,None,None,none
4,1214530451613859840,1214530451613859840,EL_chido_mai,RAZA_D_Packers,Winsconsin,GTO 🇲🇽 MkE #GOPACKGO @packers Dont take my kin...,None,{'description': {'urls': []}},False,787,...,DDEEF6,333333,True,True,True,False,None,None,None,none


In [34]:
user_df.to_pickle('user_df.pkl')